In [1]:
%load_ext autoreload
%autoreload 2

import os, glob
import functools
import pickle

import numpy as np

import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from encoders import *
from preprocessing import *
from models import *
from decoders import *
from losses import *
from callbacks import *
from metrics import *
from dataloader import make_datasets, read_violin_set
#from utils import print_plot_play, show_spectogram

In [23]:
RUN_NAME = "Supervised_Violin2"
SAMPLE_RATE = 16000
#TIME_STEPS = 1000
N_FFT = 1024
BATCH_SIZE = 32

#train_set, validation_set, _ = make_datasets(BATCH_SIZE, percent=100)
train_set, validation_set, _ = read_violin_set()
train_set

/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3])
/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3])
/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3])
/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3])
/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3])
/kuacc/users/hbalim15/.local/lib/python3.7/site-packages/librosa/core/convert.py:1354: RuntimeWarning: divide by zero encount

<PrefetchDataset shapes: {audio: (64000,), loudness_db: (1000,), f0_hz: (1000,)}, types: {audio: tf.float32, loudness_db: tf.float64, f0_hz: tf.float32}>

In [24]:
train_set = train_set.batch(32)
validation_set = validation_set.batch(32)

## Supervised Block

## TO DO

with logmag
without logmag

In [27]:
decoder = DecoderWithoutLatent(timesteps=1000)

preprocessor = F0LoudnessPreprocessor(timesteps=250)

loss = SpectralLoss(logmag_weight=1.0)
model = SupervisedAutoencoder(preprocessor=preprocessor,
                                decoder=decoder,
                                loss_fn=loss,
                                add_reverb=True)
                                
decay = ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.98)
adam = Adam(learning_rate=decay)

csv_logger = tf.keras.callbacks.CSVLogger("logs/{}.csv".format(RUN_NAME), separator=",", append=False)
callbacks = [ModelCheckpoint(model, RUN_NAME), csv_logger, CustomWandbCallback(RUN_NAME)]

model.compile(adam)

wandb: Currently logged in as: haldunbalim33 (use `wandb login --relogin` to force relogin)


In [28]:
history = model.fit(train_set, validation_data = validation_set, callbacks=callbacks, epochs=1000, batch_size=32)

Epoch 1/1000
2/2 [==============================] - 23s 5s/step - spec_loss: 15.8588 - val_spec_loss: 11.7897
Epoch 2/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 14.5546 - val_spec_loss: 11.3580
Epoch 3/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 12.2481 - val_spec_loss: 14.1383
Epoch 4/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 14.8987 - val_spec_loss: 14.3423
Epoch 5/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 14.5655 - val_spec_loss: 11.9067
Epoch 6/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 11.5032 - val_spec_loss: 13.4657
Epoch 7/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 12.7496 - val_spec_loss: 10.7955
Epoch 8/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 10.7574 - val_spec_loss: 10.7788
Epoch 9/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 11.0710 - val_spec_loss: 10.5800
Epoch 10/1000
2/2 

2/2 [==============================] - 3s 1s/step - spec_loss: 7.5537 - val_spec_loss: 8.1363
Epoch 77/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 7.5476 - val_spec_loss: 8.1213
Epoch 78/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 7.5279 - val_spec_loss: 8.1180
Epoch 79/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 7.5261 - val_spec_loss: 8.1117
Epoch 80/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 7.5209 - val_spec_loss: 8.1026
Epoch 81/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 7.5049 - val_spec_loss: 8.1032
Epoch 82/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 7.4985 - val_spec_loss: 8.0818
Epoch 83/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 7.4754 - val_spec_loss: 8.0875
Epoch 84/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 7.4727 - val_spec_loss: 8.0792
Epoch 85/1000
2/2 [=======================

2/2 [==============================] - 2s 1s/step - spec_loss: 6.8862 - val_spec_loss: 7.3503
Epoch 152/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.6681 - val_spec_loss: 7.3441
Epoch 153/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.6721 - val_spec_loss: 7.6015
Epoch 154/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.8991 - val_spec_loss: 7.4455
Epoch 155/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 6.7119 - val_spec_loss: 7.3620
Epoch 156/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.6562 - val_spec_loss: 7.4027
Epoch 157/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.7314 - val_spec_loss: 7.5136
Epoch 158/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.7489 - val_spec_loss: 7.2987
Epoch 159/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.6040 - val_spec_loss: 7.2833
Epoch 160/1000
2/2 [==============

2/2 [==============================] - 3s 1s/step - spec_loss: 6.3796 - val_spec_loss: 7.0781
Epoch 227/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.3275 - val_spec_loss: 7.0188
Epoch 228/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.3153 - val_spec_loss: 6.9982
Epoch 229/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.2828 - val_spec_loss: 7.1677
Epoch 230/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.4624 - val_spec_loss: 7.0186
Epoch 231/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.2937 - val_spec_loss: 6.9855
Epoch 232/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 6.2775 - val_spec_loss: 6.9893
Epoch 233/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.2935 - val_spec_loss: 7.0658
Epoch 234/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.3642 - val_spec_loss: 6.9579
Epoch 235/1000
2/2 [==============

2/2 [==============================] - 2s 1s/step - spec_loss: 6.1207 - val_spec_loss: 6.9301
Epoch 302/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1493 - val_spec_loss: 7.0429
Epoch 303/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 6.2761 - val_spec_loss: 6.8631
Epoch 304/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1134 - val_spec_loss: 6.8662
Epoch 305/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1172 - val_spec_loss: 7.0715
Epoch 306/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.2632 - val_spec_loss: 6.9540
Epoch 307/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1789 - val_spec_loss: 6.8790
Epoch 308/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1474 - val_spec_loss: 6.9386
Epoch 309/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 6.1404 - val_spec_loss: 6.9537
Epoch 310/1000
2/2 [==============

...

2/2 [==============================] - 2s 1s/step - spec_loss: 5.7634 - val_spec_loss: 6.6755
Epoch 677/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6658 - val_spec_loss: 6.6777
Epoch 678/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6711 - val_spec_loss: 6.6483
Epoch 679/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6245 - val_spec_loss: 6.6761
Epoch 680/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6515 - val_spec_loss: 6.6250
Epoch 681/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5771 - val_spec_loss: 6.6280
Epoch 682/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5636 - val_spec_loss: 6.6210
Epoch 683/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5531 - val_spec_loss: 6.6068
Epoch 684/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.5407 - val_spec_loss: 6.6278
Epoch 685/1000
2/2 [==============

2/2 [==============================] - 2s 1s/step - spec_loss: 5.6592 - val_spec_loss: 6.6136
Epoch 752/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6363 - val_spec_loss: 6.6337
Epoch 753/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6382 - val_spec_loss: 6.6119
Epoch 754/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6268 - val_spec_loss: 6.6310
Epoch 755/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6413 - val_spec_loss: 6.6190
Epoch 756/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6474 - val_spec_loss: 6.6557
Epoch 757/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6820 - val_spec_loss: 6.6422
Epoch 758/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6983 - val_spec_loss: 6.6717
Epoch 759/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.7278 - val_spec_loss: 6.6418
Epoch 760/1000
2/2 [==============

2/2 [==============================] - 2s 1s/step - spec_loss: 5.4559 - val_spec_loss: 6.6100
Epoch 827/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.4404 - val_spec_loss: 6.6016
Epoch 828/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.4387 - val_spec_loss: 6.5928
Epoch 829/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.4118 - val_spec_loss: 6.6006
Epoch 830/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.4030 - val_spec_loss: 6.5964
Epoch 831/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.3980 - val_spec_loss: 6.5934
Epoch 832/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.3937 - val_spec_loss: 6.5970
Epoch 833/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.3892 - val_spec_loss: 6.5918
Epoch 834/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.3902 - val_spec_loss: 6.5986
Epoch 835/1000
2/2 [==============

2/2 [==============================] - 2s 1s/step - spec_loss: 5.3886 - val_spec_loss: 6.7238
Epoch 902/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.3946 - val_spec_loss: 6.7786
Epoch 903/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.4692 - val_spec_loss: 6.8420
Epoch 904/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5165 - val_spec_loss: 6.8389
Epoch 905/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5934 - val_spec_loss: 6.8473
Epoch 906/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.6679 - val_spec_loss: 6.7394
Epoch 907/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.6437 - val_spec_loss: 6.6546
Epoch 908/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.5500 - val_spec_loss: 6.6862
Epoch 909/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.4705 - val_spec_loss: 6.7832
Epoch 910/1000
2/2 [==============

2/2 [==============================] - 2s 1s/step - spec_loss: 5.2172 - val_spec_loss: 6.6933
Epoch 977/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2206 - val_spec_loss: 6.6815
Epoch 978/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2203 - val_spec_loss: 6.6945
Epoch 979/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2053 - val_spec_loss: 6.6798
Epoch 980/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2001 - val_spec_loss: 6.7056
Epoch 981/1000
2/2 [==============================] - 3s 1s/step - spec_loss: 5.2099 - val_spec_loss: 6.6899
Epoch 982/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2154 - val_spec_loss: 6.7110
Epoch 983/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2189 - val_spec_loss: 6.6921
Epoch 984/1000
2/2 [==============================] - 2s 1s/step - spec_loss: 5.2389 - val_spec_loss: 6.7780
Epoch 985/1000
2/2 [==============

## Unsupervised Block

In [ ]:
timesteps = 250

encoder_z = Encoder_z(timesteps=timesteps)
encoder_f0 = Encoder_f(timesteps=timesteps)

decoder = DecoderWithLatent()

preprocessor = MidiF0LoudnessPreprocessor(timesteps=timesteps)

#loss = SpectralLoss(logmag_weight=1.0)
loss = MultiLoss(logmag_weight=1.0, perceptual_loss_weight=38)

metric_fns = {"F0_recons_L1": f0_midi_scaled_L1_loss}

model = UnsupervisedAutoencoder(preprocessor=preprocessor,
                                encoder_f0=encoder_f0,
                                encoder_z=encoder_z,
                                decoder=decoder,
                                loss_fn=loss,
                                tracker_names=["total_loss", "spec_loss", "perc_loss","F0_recons_L1"],
                                metric_fns=metric_fns)

decay = ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.98)
adam = Adam(learning_rate=decay)

csv_logger = tf.keras.callbacks.CSVLogger("logs/{}.csv".format(RUN_NAME), separator=",", append=False)

callbacks = [ModelCheckpoint(model, RUN_NAME), csv_logger, CustomWandbCallback(RUN_NAME)]

model.compile(adam, metrics = [f0_midi_scaled_L1_loss])

In [ ]:
history = model.fit(train_set, 
                    validation_data=validation_set,
                    callbacks=callbacks, 
                    epochs=1000,
                    steps_per_epoch=train_set.my_len,
                    validation_steps=validation_set.my_len)

# TODO:

supervised data\
supervised model

interpolation/extrapolation (optional) \
test metrics


In [ ]:
#k_filters = [128]*2 + [256]*3 + [512]*4 + [1024]*3
#s_freqs = [1,1,2]*2 + [1,1,1,2] + [1,1]



In [ ]:
model.load("SupervisedViolinModel/300/model.ckpt")

In [ ]:
from dsp_utils.spectral_ops import compute_mfcc, compute_logmel, compute_loudness, compute_f0
def calculate_recons_f0_error(dataset):
    it = iter(dataset)
    preds,truth = [],[]
    for batch in it:
        pred = model(batch)
        preds.append(pred["audio_synth"].numpy())
        truth.append(pred["inputs"]["f0_hz"].numpy())
    pred_f0 = [compute_f0(p[0], 16000, 250, viterbi=True)[0] for p in preds]
    error = np.mean(np.abs(np.array(truth)[:,0,:,0]-np.array(pred_f0)))
    hz_to_midi = core.hz_to_midi
    F0_RANGE = spectral_ops.F0_RANGE
    return hz_to_midi(error)/F0_RANGE
calculate_recons_f0_error(train_set)